### Estimate of Covid-19 Hospitalizations and Hospital Capacity

#### Aaron McAdie

I've been really curious about how the current case load in the US compares with regional hospital beds, and which regions are approaching their capacity.  The folks at the New York times just open sourced their [county-level Covid-19 case dataset](https://github.com/nytimes/covid-19-data), and it provides a great jumping off point to probe the question.

Also, since I started working on this, I came across the [incredible modeling and data viz](https://covid19.healthdata.org/projections) done by the UW IHME team.  They have done a way better job than I have the subject matter expertise to match, so I'll pull in their projections and explore it from a few different angles.

In [2]:
import requests
import re
import zipfile
from bs4 import BeautifulSoup
import pandas as pd
import altair as alt

#### Dataset 1 - NYTimes Cases, thanks NYT!

In [3]:
cases = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv')

In [4]:
cases.head()

,date,county,state,fips,cases,deaths
0,2020-01-21,Snohomish,Washington,53061.0,1,0
1,2020-01-22,Snohomish,Washington,53061.0,1,0
2,2020-01-23,Snohomish,Washington,53061.0,1,0
3,2020-01-24,Cook,Illinois,17031.0,1,0
4,2020-01-24,Snohomish,Washington,53061.0,1,0


In [5]:
cases.dtypes

date       object
county     object
state      object
fips      float64
cases       int64
deaths      int64
dtype: object

In [6]:
cases = cases.assign(
    date = pd.to_datetime(cases['date']),
    fips = cases['fips'].astype(pd.Int32Dtype())
)

Cases are cumulative, we want new cases each day to estimate the hospital case load

In [7]:
cases['cases_shifted'] = (
    cases.groupby(['county', 'state'])
    .cases
    .shift(1)
    .fillna(0)
    .astype(int)
)

In [8]:
cases['cases_new'] = cases['cases'] - cases['cases_shifted']

Check logic by plotting King County data.  I've been wanting to see more presentations of new cases per day rather than the cumulative plots that are more commonly displayed to get a better feel for how we are 'flattening the curve'.

In [9]:
king = cases.query('county == "King" & state == "Washington"')

In [82]:
base = alt.Chart(king[['date', 'cases', 'cases_new']]).encode(alt.X('monthdate(date):O', title = 'Date'))
bars = base.mark_bar(color = '#65799b').encode(y = 'cases_new', tooltip = 'cases_new')
bars.properties(title = 'King County WA Epidemiological Curve (new cases per day)')

alt.Chart(...)

In [83]:
line = base.mark_line(color = '#e23e57').encode(y = 'cases', tooltip = 'cases')

(bars + line).properties(title = 'King County Cumulative and Daily Covid-19 Cases')

alt.LayerChart(...)

Passes sanity check.  Interesting that it looks like there are 4 roughly linear regimes on the cumulative curve, just from eyeballing

In [11]:
statewide = (
    cases.groupby(['date', 'state'])
    .aggregate({'cases':'sum'})
    .reset_index()
)

In [12]:
statewide.head()

,date,state,cases
0,2020-01-21,Washington,1
1,2020-01-22,Washington,1
2,2020-01-23,Washington,1
3,2020-01-24,Illinois,1
4,2020-01-24,Washington,1


#### State by State Cases Over Time

In [14]:
(alt.Chart(statewide)
 .mark_rect()
 .encode(x = alt.X('date:O', axis = alt.Axis(labels = False)),
         y = 'state:N',
         color = 'cases:Q')
 .properties(title = 'Cumulative US Covid-19 Cases by Day', width = 600)
)

alt.Chart(...)

In [15]:
(alt.Chart(statewide[statewide['state'] != 'New York'])
 .mark_rect()
 .encode(x = alt.X('date:O', axis = alt.Axis(labels = False)),
         y = 'state:N',
         color = 'cases:Q',
         tooltip = ['date', 'state', 'cases'])
 .properties(title = 'Cumulative US Covid-19 Cases by Day (NY removed)', width = 600)
)

alt.Chart(...)

### Dataset 2 - Population and Demographics by County (U.S. Census Bureau)

In [16]:
pop = pd.read_csv('https://www2.census.gov/programs-surveys/popest/datasets/2010-2017/counties/asrh/cc-est2017-alldata.csv',
                 encoding='ISO-8859-1')

### Dataset 3 - Hospitalization Rates by Age Group (CDC)

The CDC published a table of hospitalization, ICU, and death rates by age group, based on ~2500 hospitalizations in the US in Feb/Mar.  This could be used to get a rough estimate of particular types of beds needed, namely pediatric, which is of high interest for us here at Seattle Children's.

In [21]:
# get html from CDC
cdc_html = requests.get('https://www.cdc.gov/mmwr/volumes/69/wr/mm6912e2.htm')
soup = BeautifulSoup(cdc_html.content)

In [25]:
# find table
tables = soup.find_all('table')
tables[0].find_all('tbody')

[<tbody>
 <tr>
 <td align="left" valign="top">0–19 (123)</td>
 <td align="right" valign="bottom">1.6–2.5</td>
 <td align="right" valign="bottom">0</td>
 <td align="right" valign="bottom">0</td>
 </tr>
 <tr>
 <td align="left" valign="top">20–44 (705)</td>
 <td align="right" valign="bottom">14.3–20.8</td>
 <td align="right" valign="bottom">2.0–4.2</td>
 <td align="right" valign="bottom">0.1–0.2</td>
 </tr>
 <tr>
 <td align="left" valign="top">45–54 (429)</td>
 <td align="right" valign="bottom">21.2–28.3</td>
 <td align="right" valign="bottom">5.4–10.4</td>
 <td align="right" valign="bottom">0.5–0.8</td>
 </tr>
 <tr>
 <td align="left" valign="top">55–64 (429)</td>
 <td align="right" valign="bottom">20.5–30.1</td>
 <td align="right" valign="bottom">4.7–11.2</td>
 <td align="right" valign="bottom">1.4–2.6</td>
 </tr>
 <tr>
 <td align="left" valign="top">65–74 (409)</td>
 <td align="right" valign="bottom">28.6–43.5</td>
 <td align="right" valign="bottom">8.1–18.8</td>
 <td align="right" vali

In [35]:
table = tables[0].find('tbody')

rows_dict = dict()

for idx, row in enumerate(table.find_all('tr')):
    temp_list = list()
    for col in row.find_all('td'):
        temp_list.append(col.string)
    rows_dict[idx] = temp_list

In [92]:
cdc_df = pd.DataFrame(rows_dict).T
cdc_df.columns = ['age_range', 'hosp_rate', 'icu_rate', 'death_rate']
cdc_df

,age_range,hosp_rate,icu_rate,death_rate
0,0–19 (123),1.6–2.5,0,0
1,20–44 (705),14.3–20.8,2.0–4.2,0.1–0.2
2,45–54 (429),21.2–28.3,5.4–10.4,0.5–0.8
3,55–64 (429),20.5–30.1,4.7–11.2,1.4–2.6
4,65–74 (409),28.6–43.5,8.1–18.8,2.7–4.9
5,75–84 (210),30.5–58.7,10.5–31.0,4.3–10.5
6,≥85 (144),31.3–70.3,6.3–29.0,10.4–27.3
7,"Total (2,449)",20.7–31.4,4.9–11.5,1.8–3.4


In [94]:
# clean up a bit, extract relevant info into usable columns
# self-challenge by doing this with vectorized pandas functions rather than standard regex

cdc_df = (cdc_df
          .assign(n = lambda x: x['age_range'].str.extract(r'\(([0-9,]{3,})\)'),
                  age_range = lambda x: x['age_range'].str.extract(r'(.*)\s\([0-9,]{3,}\)'),
                  hosp_lo = lambda x: x['hosp_rate'].str.extract(r'([0-9\.]{2,}).[0-9\.]{2,}'),
                  hosp_hi = lambda x: x['hosp_rate'].str.extract(r'[0-9\.]{2,}.([0-9\.]{2,})'))
         )

cdc_df

,age_range,hosp_rate,icu_rate,death_rate,n,hosp_lo,hosp_hi
0,0–19,1.6–2.5,0,0,123,1.6,2.5
1,20–44,14.3–20.8,2.0–4.2,0.1–0.2,705,14.3,20.8
2,45–54,21.2–28.3,5.4–10.4,0.5–0.8,429,21.2,28.3
3,55–64,20.5–30.1,4.7–11.2,1.4–2.6,429,20.5,30.1
4,65–74,28.6–43.5,8.1–18.8,2.7–4.9,409,28.6,43.5
5,75–84,30.5–58.7,10.5–31.0,4.3–10.5,210,30.5,58.7
6,≥85,31.3–70.3,6.3–29.0,10.4–27.3,144,31.3,70.3
7,Total,20.7–31.4,4.9–11.5,1.8–3.4,"2,449",20.7,31.4


In [95]:
# strip off total into separate df
cdc_total = cdc_df[cdc_df.age_range.str.contains('Total')]
cdc_df = cdc_df[~cdc_df.age_range.str.contains('Total')]

### Dataset 4 - Hospital Locations and Capacity (U.S. Homeland Infrastructure Foundation-Level Data)

In [27]:
tables[0].find_all('tr')

[<tr>
 <th align="left" class="row1left" rowspan="2" scope="col" valign="bottom">Age group (yrs) (no. of cases)</th>
 <th align="center" class="row1" colspan="3" scope="col" valign="bottom">%*</th>
 </tr>,
 <tr>
 <th align="center" class="row1" colspan="1" scope="col" valign="bottom">Hospitalization</th>
 <th align="center" class="row1" scope="col" valign="bottom">ICU admission</th>
 <th align="center" class="row1" scope="col" valign="bottom">Case-fatality</th>
 </tr>,
 <tr>
 <td align="left" valign="top">0–19 (123)</td>
 <td align="right" valign="bottom">1.6–2.5</td>
 <td align="right" valign="bottom">0</td>
 <td align="right" valign="bottom">0</td>
 </tr>,
 <tr>
 <td align="left" valign="top">20–44 (705)</td>
 <td align="right" valign="bottom">14.3–20.8</td>
 <td align="right" valign="bottom">2.0–4.2</td>
 <td align="right" valign="bottom">0.1–0.2</td>
 </tr>,
 <tr>
 <td align="left" valign="top">45–54 (429)</td>
 <td align="right" valign="bottom">21.2–28.3</td>
 <td align="right" va